In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.integrate
import scipy.interpolate
import numpy as np
import scipy.stats as sts
import real_TOVsolver

n0 = 0.16 #MeV/fm^3
pi = np.pi
MeV_to_km = 1.323e-6
max_p = 350 #MeV/fm^3

In [2]:
EOS_small = pd.read_table('data/EOSCEFTVE1.dat', header=None).to_numpy()

In [9]:
def sample_ns(num_points, n_start, max_n):
    
    # sample random points in density
    epsilon = 1e-17
    loc_n = n_start + epsilon #to guarentee we don't randomly pull n_start
    scale_n = max_n - n_start - epsilon
    sample_n = sts.uniform.rvs(loc=loc_n, scale=scale_n, size=num_points-1)
    # order these points
    n_sort = np.sort(sample_n)

    # construct the arrays
    n_0 = n_start*np.ones(1)
    n = np.append(n_0, n_sort)
    ns = np.append(n, max_n)
    
    return ns

def sample_cs(num_points, cs_start):
    
    # sample speeds of sound
    sample_cs = sts.uniform.rvs(size=num_points)
    
    c_0 = cs_start*np.ones(1)
    cs = np.append(c_0, sample_cs)
    
    return ns, cs

def extend_cs(n_step, starts, cs2_func):

    size = int((max_n-n_start)/n_step)

    # initialize array
    EOS_ex = np.zeros((size, 3))
    # set starting values at n = 2n0
    EOS_ex[0,0] = starts[0]
    EOS_ex[0,1] = starts[1]
    EOS_ex[0,2] = starts[2]

    for k in range(size-1):
        # n_i+1
        EOS_ex[k+1,0] = EOS_ex[k,0] + n_step
        # p_i+1
        if cs2_func(EOS_ex[k,0]) > 1:
            EOS_ex[k+1,1] = EOS_ex[k,1] + n_step*((EOS_ex[k,1]+EOS_ex[k,2])/EOS_ex[k,0])
        else:
            EOS_ex[k+1,1] = EOS_ex[k,1] + n_step*(cs2_func(EOS_ex[k,0]))*((EOS_ex[k,1]+EOS_ex[k,2])/EOS_ex[k,0])
        # e_i+1
        EOS_ex[k+1,2] = EOS_ex[k,2] + n_step*((EOS_ex[k,1]+EOS_ex[k,2])/EOS_ex[k,0])
        
    return EOS_ex

def sample_P(num_points, p_start, max_p):
    
    # sample pressures
    epsilon_p = 1e-7*p_start
    loc_p = p_start + epsilon_p #to guarentee we don't randomly pull p_start
    scale_p = max_p - p_start - epsilon_p
    sample_p = sts.uniform.rvs(loc=loc_p, scale=scale_p, size=num_points-1)
    # order these points
    p_sort = np.sort(sample_p)

    p_0 = p_start*np.ones(1)
    p = np.append(p_0, p_sort)
    ps = np.append(p, max_p)
    
    return ps

def extend_Plin(n_step, starts, P_lin):

    size = int((max_n-n_start)/n_step)

    # initialize array
    EOS_ex = np.zeros((size, 3))
    # set starting values at n = 2n0
    EOS_ex[0,0] = starts[0]
    EOS_ex[0,1] = starts[1]
    EOS_ex[0,2] = starts[2]

    for k in range(size-1):
        # n_i+1
        EOS_ex[k+1,0] = EOS_ex[k,0] + n_step
        # p_i+1
        EOS_ex[k+1,1] = P_lin(EOS_ex[k+1,0])
        # e_i+1
        EOS_ex[k+1,2] = EOS_ex[k,2] + n_step*((EOS_ex[k,1]+EOS_ex[k,2])/EOS_ex[k,0])
        
    return EOS_ex

def sample_polytrop(num_points, ns, p_start, a_gamma):
    n_0 = ns[0]*np.ones(1)
    
    # sample random gammas
    gammas = sts.gamma.rvs(a=a_gamma, size=num_points+1)
    
    # initialize K array
    K0 = p_start*(n_0**(-gammas[0]))
    K = np.zeros(num_points)
    Ks = np.append(K0, K)
    
    for i in range(num_points):
        Ks[i+1] = Ks[i]*(ns[i+1]**(gammas[i]-gammas[i+1]))
        
    return gammas, Ks

def extend_EOS_polytrop(n_step, ns, starts, Ks, gammas):

    size = int((ns[-1]-ns[0])/n_step)

    # initialize array
    EOS_ex = np.zeros((size, 3))
    # set starting values at n = 2n0
    EOS_ex[0,0] = starts[0]
    EOS_ex[0,1] = starts[1]
    EOS_ex[0,2] = starts[2]
    
    i=0

    for k in range(size-1):
        # n_i+1
        EOS_ex[k+1,0] = EOS_ex[k,0] + n_step
        # p_i+1
        if ns[i] < EOS_ex[k+1,0] < ns[i+1]:
            EOS_ex[k+1,1] = EOS_ex[k,1] + n_step*(Ks[i]*gammas[i]*(EOS_ex[k,0]**(gammas[i])))
        else:
            i+=1
            EOS_ex[k+1,1] = EOS_ex[k,1] + n_step*(Ks[i]*gammas[i]*(EOS_ex[k,0]**(gammas[i])))
        # e_i+1
        EOS_ex[k+1,2] = EOS_ex[k,2] + n_step*((EOS_ex[k,1]+EOS_ex[k,2])/EOS_ex[k,0])
        
    return EOS_ex

def stitch_EOS(small_EOS, EOS_ex):
    
    # get relevant sizes
    size_smol = small_EOS.shape[0] -1 # -1 becuase we don't want last duplicated entry
    size_ex = EOS_ex.shape[0]
    
    # initialize array
    tot_EOS = np.zeros((size_smol+size_ex, small_EOS.shape[1]))
    
    tot_EOS[:size_smol,:] = small_EOS[:size_smol,:]
    tot_EOS[size_smol:,0] = EOS_ex[:,0]
    tot_EOS[size_smol:,1] = EOS_ex[:,1]
    tot_EOS[size_smol:,2] = EOS_ex[:,2]
    
    return tot_EOS

In [6]:
def simulate_EOS(nsamp, small_EOS, rho_step, rho_max, ext_type, a_gamma):
    """
    Function to simulate and extend an EOS three ways
    
    inputs
    nsim: number of extensions to produce
    nsamp: number of density points to sample when making extentions
    small_EOS: the low density EOS to be extended
    rho_step: the step size in rho when extending
    rho_max: the maximum density of extension
    ext_type: the type of extension to do (speed of sound, linear P, and polytropic)
        cs (string) or 1 (int)
        linear or 2
        polytrop or 3
    a_gamma: only needed if doing polytropic extension, but characterized the distribution from
                which gamma's are drawn
                
    output
    EOS_tot = extended EOS stiched after small_EOS
    """
    n = small_EOS[:,0] 
    p = small_EOS[:,1] 
    e = small_EOS[:,2] 
    
    starts = [n[-1], p[-1], e[-1]]
    
    n0 = 0.16 #MeV/fm^3
    max_p = 350 #MeV/fm^3

    ns = sample_ns(nsamp, n[-1], rho_max)

    if ext_type == 'cs' or ext_type == 1:
        # derivative of pressure wrt energy
        dp_de = scipy.interpolate.CubicSpline(p, e).derivative(nu=1)
        
        #definition of speed of sound
        cs_start = np.sqrt(1/dp_de(p[-1]))
        
        #sample cs
        cs = sample_cs(nsamp, cs_start)
        
        # make speed of sound squared function
        cs2_func = scipy.interpolate.interp1d(ns, cs**2)
        
        # extend EOS
        EOS_ex = extend_EOS_cs(rho_step, starts, cs2_func)
        
    elif ext_type == 'linear' or ext_type == 2:
        # sample P
        ps = sample_P(nsamp, p[-1], max_p)
        
        # make linear interpolation
        P_lin = scipy.interpolate.interp1d(ns, ps)
        
        # extend EOS
        EOS_ex = extend_Plin(rho_step, starts, P_lin)
        
    elif ext_type == 'polytrop' or ext_type == 3:
        # sample parameters
        gammas, Ks = sample_polytrop(nsamp, ns, p[-1], a_gamma)
        
        # extend EOS
        EOS_ex = extend_EOS_polytrop(rho_step, ns, starts, Ks, gammas)
        
    else:
        print('The ext_type you entered does not match any of the allowed types. Please enter "cs" for a ' +
              'speed of sound extension, "linear" for an extension linear in pressure and "polytop" ' +
             'for a polytropic extension')
        
    EOS_tot = stitch_EOS(small_EOS, EOS_ex)
    
    return EOS_tot

In [18]:
nsamp = 5
rho_step = 1e-3
rho_max = 10*n0
a_gamma = 5/3

EOS_tot = simulate_EOS(nsamp, EOS_small, rho_step, rho_max, ext_type, a_gamma)

MRL_test = real_TOVsolver.solve(EOS_tot, max_p)

/opt/anaconda3/lib/python3.8/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [19]:
nsim = 50
shape_EOS = (EOS_tot.shape[0], EOS_tot.shape[1], nsim)
shape_MRL = (MRL_test.shape[0],MRL_test.shape[1], nsim)
print(MRL_test.shape)

(100, 3)


In [20]:
many_EOS_poly = np.zeros(shape_EOS)
many_MRL_poly = np.zeros(shape_MRL)
for i in range(nsim):
    many_EOS_poly[:,:,i] = simulate_EOS(nsamp, EOS_small, rho_step, rho_max, 'polytrop', a_gamma)
    many_MRL_poly[:,:,i] = real_TOVsolver.solve(many_EOS_poly[:,:,i], max_p)

/Users/brendanking/Documents/LANL_project/real_TOVsolver.py:51: RuntimeWarning: invalid value encountered in log
  den = 2 * C * a + 4 * (C ** 3) * b + 3 * ((1 - 2 * C) ** 2) * c * np.log(1 - 2 * C)
/Users/brendanking/Documents/LANL_project/real_TOVsolver.py:51: RuntimeWarning: overflow encountered in double_scalars
  den = 2 * C * a + 4 * (C ** 3) * b + 3 * ((1 - 2 * C) ** 2) * c * np.log(1 - 2 * C)
/Users/brendanking/Documents/LANL_project/real_TOVsolver.py:53: RuntimeWarning: overflow encountered in double_scalars
  return (8 * (C ** 5) * num) / (5 * den)


KeyboardInterrupt: 

In [ ]:
for i in range(nsim):
    plot.plot(many_MRL_poly[:,0,i], many_MRL_poly[:,1,i])

In [ ]:
many_EOS_cs = np.zeros(shape_EOS)
many_MRL_cs = np.zeros(shape_MRL)
for i in range(nsim):
    many_EOS_cs[:,:,i] = simulate_EOS(nsamp, EOS_small, rho_step, rho_max, 'cs', a_gamma)
    many_MRL_cs[:,:,i] = real_TOVsolver.solve(many_EOS_cs[:,:,i], max_p)

In [ ]:
many_EOS_lin = np.zeros(shape_EOS)
many_MRL_lin = np.zeros(shape_MRL)
for i in range(nsim):
    many_EOS_lin[:,:,i] = simulate_EOS(nsamp, EOS_small, rho_step, rho_max, 'linear', a_gamma)
    many_MRL_lin[:,:,i] = real_TOVsolver.solve(many_EOS_lin[:,:,i], max_p)